The aim of this predictive model is to predict if a certain person will receive a bank loan or not, using machine learning and deep learning tools.

In [1]:
# importations sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score, validation_curve, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [2]:
# importations pytorch
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader                    # to build data set with pytorch
import torch.nn as nn 
from torch.nn import Module                                                             # neural network
import torch.optim as optim
from torch.optim import Optimizer
from torch.nn.modules.loss import _Loss
import torchmetrics
from torchmetrics import Metric


In [3]:
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


C:\Users\USER\AppData\Local\Temp\ipykernel_13708\918415353.py:1: DtypeWarning: Columns (36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


In [4]:
head=dataset.head()
info=dataset.info()
print(head)
print(info)                                                                         # 28632 rows and 78 columns, there are 3 Dtypes ( float, int and object=[string] )
target_name= "action_taken_name"                                                    # purpose name
data , target = dataset.drop(columns=target_name), dataset[target_name]             # data to browse and target"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   as_of_year                      28632 non-null  int64  
 1   respondent_id                   28632 non-null  object 
 2   agency_name                     28632 non-null  object 
 3   agency_abbr                     28632 non-null  object 
 4   agency_code                     28632 non-null  int64  
 5   loan_type_name                  28632 non-null  object 
 6   loan_type                       28632 non-null  int64  
 7   property_type_name              28632 non-null  object 
 8   property_type                   28632 non-null  int64  
 9   loan_purpose_name               28632 non-null  object 
 10  loan_purpose                    28632 non-null  int64  
 11  owner_occupancy_name            28632 non-null  object 
 12  owner_occupancy                 

In [5]:
#c. Feature Engineering
#Rien d'implémenter de connu pour faire ce que l'on doit faire. 
#Le gros du travail qui va être fait: 

#Lien nécessaire entre eric et jomuel
dataset = data

#Eléments à supprimer: 

#On va enlever les éléments qui ont des cases  vides dans: 
#C'est que ce sont des logements saisonniers/temporaires dans une zone de l'alaska sous sous peuplé.
#Que 75000 habitants sur plusieurs millions. 
#Sinon on perd des données c'est dommage et si on les remplace par des 0 cela risque de mal influencer notre modèle. 
#Car on perd plein d'autres données. 
#On enlève donc : ceux qui n'ont pas de county_code 
#et ceux qui n'ont pas:   applicant_income_000s
#On doit supprimer les éléments qui ont vide pour : applicant_income_000s et county_code 
#On crée L liste des éléments à supprimer:
L=[]
#pour county_code
#on va créer un dataset composé uniquement de county_code
#Basée sur la colonne "county_code": 
df_county = pd.DataFrame({"dataset county code":dataset["county_code"]})
#on va en récupérer toutes les lignes nulles
l_nulles_county = df_county.isnull().values.any()
#et les ajouter à L
L+=l_nulles_county
#pour applicant_income_000s
#on va créer un dataset composé uniquement de applicant_income_000s
#Basée sur la colonne "applicant_income_000s": 
df_income = pd.DataFrame({"dataset applicant_income_000s":dataset["applicant_income_000s"]})
#on va en récupérer toutes les lignes nulles
l_nulles_income = df_income.isnull().values.any()
#et les ajouter à L
L+=l_nulles_income
#suppression des lignes de notre liste:
dataset = dataset.drop(index=L)


#les colonnes dont nous devons nous occuper:

#les colonnes à supprimer: on va en récupérer les noms 
#on va récupérer les inutiles et celles qui forment des doublons. Par exemple on a des colonnes dont on a du data 
#categorical et ensuite sa transcription en numerical. On garde alors juste la transcription en numerical
#Les colonnes que l'on enlève et l'explication qui va avec: 
# Les inutiles
#la première: as_of_year, respondent_id,  state_abbr, state_code, hoepa_status, edit_status, sequence_number,
#application_date_indicator
dataset= dataset.drop(columns=["as_of_year", "respondent_id",  "state_abbr", "state_code", "hoepa_status", "edit_status", "sequence_number"])
#et aussi: 
#denial_reason_1, denial_reason_2, denial_reason_3 et rate_spread
#car beaucoup trop de données manquantes
dataset = dataset.drop(columns=["denial_reason_1", "denial_reason_2", "denial_reason_3", "rate_spread"])

#Toutes celles où l'on a la transposition en numerical ensuite:
#On garde toutes les colonnes qui sont en doublons avec les colonnes _name.
#Car on va toutes les traiter avec OneHotEncoder plus simplement
# On identifie toutes les colonnes qui finissent par "_name"
name_cols = [col for col in dataset.columns if col.endswith("_name")]
# On identifie alors toutes les colonnes qui correspondent sont les transpositions
#en numerical des colonnes qui finissent par name. 
radicals = [col[:-5] for col in name_cols]
# On récupère alors les numéros de colonnes auquels correspondent ces colonnes
cols_to_drop = [col for col in dataset.columns if col in radicals]
#Et on enlève toutes ces colonnes
dataset=dataset.drop(columns=cols_to_drop)
#que nous n'ayons plus que des données numerical:
#On applique OneHotEncoder quand il faut. 
"""pour la race de l'applicant notamment: 
On va utiliser oneHotencoder
On ne va pas faire comme dans le dataset. On va créer 6 nouvelles features.
Car on a quand même plus de 500 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 
n avait tout coder à la main avant de découvrir one hot encoder. 
On fait pareil pour la race du co applicant. Mais cette fois ci avec 8 catégories. 
On ne va pas faire comme dans le dataset. On va créer 8 nouvelles features.
Car on a quand même plus de 150 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 
Et sinon bien sur on met un 1 dans notre provided. """
#On sélectionne les numerical features
numerical_features=dataset.select_dtypes(include='number').columns.tolist()                         # stockage des données numériques (13 colonnes)
#On en déduit les categorical features
categorical_features=list(set(dataset.columns)-set(numerical_features))                             # stockage des données catégoriques(18 colonnes)
#On va maintenant faire le preprocessing: 
#On définit la pipeline de preprocessing
numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
preprocessor = ColumnTransformer(transformers=[("numeric",numeric, numerical_features),
("categoric",categoric, categorical_features)])
#et on l'applique sur les datas
#data preparation
X_full=dataset[numerical_features+categorical_features]
#data_clean = pd.DataFrame({"dataset final": X_full})
#application
dataset=preprocessor.fit_transform(X_full)
X_full = dataset
#On renomme X_full qui n'est plus un dataset on le rappelle. 
data_clean=X_full

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [6]:

# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('rfc', RandomForestClassifier())
])

# data preparation
Y= target

# training
model_pipeline_full.fit(X_full,Y)

# predictive model
y_pred=model_pipeline_full.predict(X_full)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    X_full,
    Y,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

Accuracy_full  (10-fold CV) : 0.98


In [7]:
print(pd.Series(Y).value_counts())   # occurrences


action_taken_name
Loan originated                                14430
Loan purchased by the institution               6415
Application withdrawn by applicant              3495
Application denied by financial institution     3075
File closed for incompleteness                   756
Application approved but not accepted            461
Name: count, dtype: int64


Let's Build and train a classification MLP on our data  with pytorch tools.

Step 1: Load the dataset using sklearn and create a custom PyTorch Dataset for this data.

In [8]:
# implementation of the MLP

encoder=LabelEncoder()                                                                  # instance encoder
target_encoded=encoder.fit_transform(target)                                            # target encoding for tensor management

#X_prepared=preprocessor.fit_transform(X_full)                                           # prepares X_full for pytorch( in numpy)

X=torch.tensor(X_full,dtype=torch.float32)                                          # data tensor building

means = X.mean(dim=0, keepdim=True)                                                     # mean
stds = X.std(dim=0, keepdim=True)                                                       # standard deviation(sigma)
 
stds[stds == 0] = 1.0                                                                   # avoid the 0-division 

X_standardized= (X - means) / stds                                                      # scaling

y=torch.tensor(target_encoded, dtype=torch.long)                                        # target tensor building                                           

data_set_torch=TensorDataset(X_standardized,y)                                          # data set building 

sample0, target0 = data_set_torch[0]
print (sample0.shape, target0.shape)                                                    # on obtient des vecteurs de dimensiosn 328 et le tzrget etant scalaore 

torch.Size([196]) torch.Size([])



Step 2: Create data loaders for training, validation, and testing.

In [9]:
torch.manual_seed(42)

# define sizes for each step of the learning
train_size=len(data_set_torch)*80//100
valid_size=len(data_set_torch)*20//100
test_size=len(data_set_torch) - (train_size + valid_size)
train_dataset, valid_dataset , test_dataset = random_split(           
                                data_set_torch,
                                [train_size,valid_size,test_size]
                                ) 
# data loaders (to load data)
batch_size=32                                       # because it's a cpu
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
valid_loader=DataLoader(valid_dataset,batch_size=batch_size)
test_loader=DataLoader(test_dataset,batch_size=batch_size)


Step 3: Let's build a custom MLP module to tackle our classification task

In [10]:
# let's start by the sequential way (most obvious)
X_tensor, y_tensor=data_set_torch.tensors
n_inputs= X_tensor.shape[1]
n_classes= len(torch.unique(y_tensor))                      # number of differents categories to predict  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set the model
bank_loan_model = nn.Sequential(                       
    nn.Linear(n_inputs,200), nn.ReLU(),                    # linear layer , ReLU activation      
    nn.Linear(200,100), nn.ReLU(),
    nn.Linear(100,50), nn.ReLU(),
    nn.Linear(50, n_classes)
    ).to(device)                                           # puts on the cpu


In [11]:
     # let's build our training
n_epochs = 30
learning_rate= [0.01, 0.001 , 0.0001]

def train(
        
        model:Module,
        optimizer: Optimizer,
        criterion: _Loss ,                                                                            # criteria
        metric: Metric,                                                                               # metric
        train_loader: DataLoader , 
        valid_loader: DataLoader ,
        n_epochs:int,
        device: str = "cpu"
       
        )  :
    
        history={ "train_loss":[] , "train_metric":[], "valid_metric": []    }                             # initilization of the output

    

        for epoch in range(n_epochs):
            
            model.train()                                                                                  # sets the model in a training mode     
            train_loss=0.0                                                                                 # initialization
            metric.reset()

            for X_batch , y_batch in train_loader :
                X_batch, y_batch = X_batch.to(device),y_batch.to(device)                                   # sets on the device 
                y_pred=model(X_batch)
                loss = criterion(y_pred,y_batch)
                loss.backward()                                                                            # back propagation
                optimizer.step()                                                                           # updates model parameters
                optimizer.zero_grad()                                                                      # renitialize gradient to zero to avoid accumulation
                train_loss+= loss.item()       
                metric.update(y_pred,y_batch)                                                              # performance metric       

            train_loss/=len(train_loader)
            train_metric=metric.compute().item()

            # validation step

            model.eval()                                                                                     # set the model in a validation mode
            metric.reset()
            with torch.no_grad():                                                                            # avoid to compute gradient
                for X_val, y_val in valid_loader:
                    X_val, y_val = X_val.to(device), y_val.to(device)
                    outputs = model(X_val)
                    metric.update(outputs, y_val)

            valid_metric = metric.compute().item()


            history["train_loss"].append(train_loss)
            history["train_metric"].append(train_metric)
            history["valid_metric"].append(valid_metric)

        return history



# let's train our model
for lr in learning_rate:

    optimizer= torch.optim.SGD(bank_loan_model.parameters(), lr=lr)                            # optimizer with stochastic gradient descent
    criterion= nn.CrossEntropyLoss ()       
    metric= torchmetrics.Accuracy(task="multiclass",num_classes=n_classes).to(device)    

# out put

train(bank_loan_model, optimizer, criterion, metric, train_loader,                          # return the output of the training
                    valid_loader, n_epochs)




{'train_loss': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'train_metric': [0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171,
  0.01685221493244171],
 'valid_metric': [0.013098148629069328,
  0.013098148629069328,
  0.

In [12]:
# test step with test_loader

def evaluate(
    model: Module,
    criterion: _Loss,
    test_loader: DataLoader,
    device: str = "cpu",
    aggregate_fn=torch.mean

    ):

    model.eval()          
    metrics=[]    

    with torch.no_grad(): 
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            metrics.append(loss)
            

            
    return aggregate_fn(torch.stack(metrics))


evaluate(bank_loan_model,criterion=nn.CrossEntropyLoss(), test_loader=test_loader )

tensor(nan)